# Proposal 3

$$
\mathcal{L} = \mathcal{L}_c + \lambda_e \cdot || \bm{p^*_A} \odot \bm{(\theta - \theta^*_A) }||_1 + \lambda_f \cdot || \bm{(1 - p)} \odot \bm{\theta} ||_1
$$

In [1]:
%load_ext autoreload
%autoreload 2

# Libraries

In [2]:
import pickle
import pandas as pd
import plotly.express as px
from tqdm import tqdm

In [3]:
from avalanche.benchmarks.classic import PermutedMNIST
from avalanche.models import SimpleMLP
from avalanche.training.plugins import EvaluationPlugin
from avalanche.logging import InteractiveLogger
from avalanche.evaluation.metrics import (
    accuracy_metrics,
    bwt_metrics,
    confusion_matrix_metrics,
    forgetting_metrics,
    timing_metrics,
    cpu_usage_metrics,
    gpu_usage_metrics,
    disk_usage_metrics,
    ram_usage_metrics,
    loss_metrics,
    class_accuracy_metrics,
    accuracy_matrix_metrics
)

from torch.nn import CrossEntropyLoss
from torch.optim import SGD
from torch import cuda

/Users/jesusoyanedelespinoza/.pyenv/versions/3.9.18/envs/tt2/lib/python3.9/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


## Custom Libraries

In [4]:
import sys

sys.path.append("../base_code/")

from base_code.constants import DATASETS_PATH, SAVED_METRICS_PATH
from base_code.training import MWUNV1
from base_code.plugins import WeightStoragePlugin

# Constants

In [5]:
TORCH_DEVICE = "cuda" if cuda.is_available() else "cpu"
N_EXPERIENCES = 50
EXPERIMENT_SEED = 1234
METRICS_SUBFOLDER = f"class_saturation/{N_EXPERIENCES}"
METRICS_FILENAME = f"{METRICS_SUBFOLDER}/proposal_3.pkl"

# Dataset and definitions

## Dataset loading

We load state-of-the-art dataset Modified NIST

In [6]:
scenario = PermutedMNIST(N_EXPERIENCES, seed=EXPERIMENT_SEED, dataset_root=DATASETS_PATH)

## Scenario creation with train test streaming

In this point, we define our scenario considering a training where in every experience of it, a new class is presented. This is, first we train with a class $a$, the following experience we train with class $b$ ($a \neq b$)

In [7]:
train_stream = scenario.train_stream
test_stream = scenario.test_stream

In [8]:
best_hyperparameters = {
    'lambda_e': 10.0,
    'lambda_f': 10.0,
    'eps': 0.0001
}

## Evaluation metrics definition

In [9]:
eval_plugin = EvaluationPlugin(
    accuracy_metrics(experience=True, stream=True),
    forgetting_metrics(experience=True, stream=True),
    bwt_metrics(experience=True, stream=True),

    loggers=[InteractiveLogger()]
)

## Model, Optimizer, Loss function and Strategy definition

* `model`: Multi Layer Perceptron
* `Optimizer`: Adam
* `Loss function`: Cross Entropy
* `Strategy`: Elastic Weight Consolidation

In [10]:
model = SimpleMLP(num_classes=scenario.n_classes, input_size=28 * 28, hidden_layers=2, hidden_size=100).to(TORCH_DEVICE)
optimizer = SGD(model.parameters(), lr=1e-3)
criterion = CrossEntropyLoss().to(TORCH_DEVICE)
strategy = MWUNV1(
    model,
    optimizer,
    criterion,
    train_epochs=5,
    train_mb_size=128,
    evaluator=eval_plugin,
    eval_mb_size=128,
    device=TORCH_DEVICE,
    **best_hyperparameters
)

# Training and evaluation

Revisar porque el entrenamiento se está comportando de forma rara

In [11]:
for experience in tqdm(train_stream):
    strategy.train(experience)
    strategy.eval(test_stream)

  0%|          | 0/50 [00:00<?, ?it/s]

-- >> Start of training phase << --
100%|██████████| 469/469 [00:04<00:00, 100.56it/s]
Epoch 0 ended.
100%|██████████| 469/469 [00:04<00:00, 103.42it/s]
Epoch 1 ended.
100%|██████████| 469/469 [00:04<00:00, 102.28it/s]
Epoch 2 ended.
100%|██████████| 469/469 [00:04<00:00, 99.42it/s] 
Epoch 3 ended.
100%|██████████| 469/469 [00:04<00:00, 104.51it/s]
Epoch 4 ended.
-- >> End of training phase << --
-- >> Start of eval phase << --
-- Starting eval on experience 0 (Task 0) from test stream --
100%|██████████| 79/79 [00:00<00:00, 129.71it/s]
> Eval on experience 0 (Task 0) from test stream ended.
	Top1_Acc_Exp/eval_phase/test_stream/Task000/Exp000 = 0.7371
-- Starting eval on experience 1 (Task 0) from test stream --
100%|██████████| 79/79 [00:00<00:00, 125.21it/s]
> Eval on experience 1 (Task 0) from test stream ended.
	Top1_Acc_Exp/eval_phase/test_stream/Task000/Exp001 = 0.2233
-- Starting eval on experience 2 (Task 0) from test stream --
100%|██████████| 79/79 [00:00<00:00, 134.50it/s]
>

  2%|▏         | 1/50 [00:52<42:54, 52.53s/it]


> Eval on experience 49 (Task 0) from test stream ended.
	Top1_Acc_Exp/eval_phase/test_stream/Task000/Exp049 = 0.1387
-- >> End of eval phase << --
	StreamBWT/eval_phase/test_stream = 0.0000
	StreamForgetting/eval_phase/test_stream = 0.0000
	Top1_Acc_Stream/eval_phase/test_stream/Task000 = 0.1749
-- >> Start of training phase << --
100%|██████████| 469/469 [00:04<00:00, 107.20it/s]
Epoch 0 ended.
100%|██████████| 469/469 [00:04<00:00, 108.37it/s]
Epoch 1 ended.
100%|██████████| 469/469 [00:04<00:00, 104.26it/s]
Epoch 2 ended.
100%|██████████| 469/469 [00:04<00:00, 108.63it/s]
Epoch 3 ended.
100%|██████████| 469/469 [00:04<00:00, 108.14it/s]
Epoch 4 ended.
-- >> End of training phase << --
-- >> Start of eval phase << --
-- Starting eval on experience 0 (Task 0) from test stream --
100%|██████████| 79/79 [00:00<00:00, 142.04it/s]
> Eval on experience 0 (Task 0) from test stream ended.
	ExperienceBWT/eval_phase/test_stream/Task000/Exp000 = -0.0127
	ExperienceForgetting/eval_phase/test_s

  4%|▍         | 2/50 [01:43<41:17, 51.62s/it]

...


> Eval on experience 49 (Task 0) from test stream ended.
	Top1_Acc_Exp/eval_phase/test_stream/Task000/Exp049 = 0.1044
-- >> End of eval phase << --
	StreamBWT/eval_phase/test_stream = -0.3370
	StreamForgetting/eval_phase/test_stream = 0.3370
	Top1_Acc_Stream/eval_phase/test_stream/Task000 = 0.4737
-- >> Start of training phase << --
100%|██████████| 469/469 [00:04<00:00, 102.57it/s]
Epoch 0 ended.
100%|██████████| 469/469 [00:04<00:00, 103.87it/s]
Epoch 1 ended.
100%|██████████| 469/469 [00:04<00:00, 105.82it/s]
Epoch 2 ended.
100%|██████████| 469/469 [00:04<00:00, 100.28it/s]
Epoch 3 ended.
100%|██████████| 469/469 [00:04<00:00, 104.88it/s]
Epoch 4 ended.
-- >> End of training phase << --
-- >> Start of eval phase << --
-- Starting eval on experience 0 (Task 0) from test stream --
100%|██████████| 79/79 [00:00<00:00, 129.40it/s]
> Eval on experience 0 (Task 0) from test stream ended.
	ExperienceBWT/eval_phase/test_stream/Task000/Exp000 = -0.5366
	ExperienceForgetting/eval_phase/test_

 92%|█████████▏| 46/50 [40:52<03:38, 54.74s/it]


> Eval on experience 49 (Task 0) from test stream ended.
	Top1_Acc_Exp/eval_phase/test_stream/Task000/Exp049 = 0.0923
-- >> End of eval phase << --
	StreamBWT/eval_phase/test_stream = -0.3349
	StreamForgetting/eval_phase/test_stream = 0.3349
	Top1_Acc_Stream/eval_phase/test_stream/Task000 = 0.4836
-- >> Start of training phase << --
100%|██████████| 469/469 [00:05<00:00, 84.75it/s]
Epoch 0 ended.
100%|██████████| 469/469 [00:04<00:00, 94.57it/s] 
Epoch 1 ended.
100%|██████████| 469/469 [00:05<00:00, 90.32it/s] 
Epoch 2 ended.
100%|██████████| 469/469 [00:05<00:00, 89.77it/s] 
Epoch 3 ended.
100%|██████████| 469/469 [00:04<00:00, 100.64it/s]
Epoch 4 ended.
-- >> End of training phase << --
-- >> Start of eval phase << --
-- Starting eval on experience 0 (Task 0) from test stream --
100%|██████████| 79/79 [00:00<00:00, 130.74it/s]
> Eval on experience 0 (Task 0) from test stream ended.
	ExperienceBWT/eval_phase/test_stream/Task000/Exp000 = -0.5190
	ExperienceForgetting/eval_phase/test_s

 94%|█████████▍| 47/50 [41:50<02:46, 55.56s/it]


> Eval on experience 49 (Task 0) from test stream ended.
	Top1_Acc_Exp/eval_phase/test_stream/Task000/Exp049 = 0.1113
-- >> End of eval phase << --
	StreamBWT/eval_phase/test_stream = -0.3447
	StreamForgetting/eval_phase/test_stream = 0.3447
	Top1_Acc_Stream/eval_phase/test_stream/Task000 = 0.4823
-- >> Start of training phase << --
100%|██████████| 469/469 [00:05<00:00, 89.56it/s] 
Epoch 0 ended.
100%|██████████| 469/469 [00:04<00:00, 97.82it/s] 
Epoch 1 ended.
100%|██████████| 469/469 [00:05<00:00, 92.86it/s] 
Epoch 2 ended.
100%|██████████| 469/469 [00:05<00:00, 85.61it/s] 
Epoch 3 ended.
100%|██████████| 469/469 [00:04<00:00, 98.13it/s] 
Epoch 4 ended.
-- >> End of training phase << --
-- >> Start of eval phase << --
-- Starting eval on experience 0 (Task 0) from test stream --
100%|██████████| 79/79 [00:00<00:00, 133.05it/s]
> Eval on experience 0 (Task 0) from test stream ended.
	ExperienceBWT/eval_phase/test_stream/Task000/Exp000 = -0.5143
	ExperienceForgetting/eval_phase/test_

# Get metrics

In [ ]:
accuracies: dict[int, list[float]] = dict()
forgettings: dict[int, list[float]] = dict()
bwt: dict[int, list[float]] = dict()
class_accuracies: dict[int, list[float]] = dict()
losses: dict[int, list[float]] = dict()

for i in range(N_EXPERIENCES):
    filled_i = str(i).zfill(3)
    accuracies[f"Task{i}"] = eval_plugin.get_all_metrics()[
        f"Top1_Acc_Exp/eval_phase/test_stream/Task000/Exp{filled_i}"
    ][1]
    forgettings[f"Task{i}"] = eval_plugin.get_all_metrics()[
        f"ExperienceForgetting/eval_phase/test_stream/Task000/Exp{filled_i}"
    ][1]
    bwt[f"Task{i}"] = eval_plugin.get_all_metrics()[
        f"ExperienceBWT/eval_phase/test_stream/Task000/Exp{filled_i}"
    ][1]
        

accuracies["Overall"] = eval_plugin.get_all_metrics()[
    "Top1_Acc_Stream/eval_phase/test_stream/Task000"
][1]

# Store metrics

In [ ]:
pickle.dump({
    "accuracies": accuracies,
    "forgettings": forgettings,
    "bwt": bwt,
}, open(SAVED_METRICS_PATH / METRICS_FILENAME, "wb"))